## **0. Installer les librairies**

In [ ]:
!pip install folium geopandas pandas shapely
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString
from shapely.ops import unary_union

# **1. Prétraitement des données pour TC (première séléction de POI)**

5.B.1. Prétraitement des données de GTFS et Calcul de temps du TC (dans le mémoire)

## Télécharger les données GTFS

In [ ]:
stop_times = pd.read_csv('stop_times.txt', dtype={'arrival_time': str, 'departure_time': str})
stops = pd.read_csv('stops.txt')
trips = pd.read_csv('trips.txt')
routes = pd.read_csv('routes.txt')

## Calculer le temps du trajet

In [ ]:
# hh:mm:ss  >> temps de trajet
# Conversion des horaires texte (hh:mm:ss) en secondes
def to_seconds(t):
    try:
        h, m, s = map(int, t.split(':'))
        return h*3600 + m*60 + s
    except:
        return None

stop_times['arrival_seconds'] = stop_times['arrival_time'].apply(to_seconds)
stop_times['departure_seconds'] = stop_times['departure_time'].apply(to_seconds)

# Ordre des arrêts et calcul du temps entre deux arrêts
stop_times = stop_times.sort_values(['trip_id', 'stop_sequence'])
stop_times['next_stop_id'] = stop_times.groupby('trip_id')['stop_id'].shift(-1)
stop_times['next_arrival_seconds'] = stop_times.groupby('trip_id')['arrival_seconds'].shift(-1)
stop_times['travel_time'] = stop_times['next_arrival_seconds'] - stop_times['departure_seconds']

# Suppression de ligne non valide
stop_times = stop_times.dropna(subset=['next_stop_id', 'travel_time'])

# Jointure avec les fichiers trips, routes et stops
# trip → route → type
stop_times = stop_times.merge(trips[['trip_id', 'route_id']], on='trip_id', how='left')
stop_times = stop_times.merge(routes[['route_id', 'route_type']], on='route_id', how='left')

# Ajouter le nom de la station（from, to） Pour connaître le type de transport (bus, tram, etc.)
stop_times = stop_times.merge(stops[['stop_id', 'stop_name']], on='stop_id', how='left')
stop_times = stop_times.rename(columns={'stop_name': 'from_stop_name'})

stop_times = stop_times.merge(stops[['stop_id', 'stop_name']], left_on='next_stop_id', right_on='stop_id', how='left')
stop_times = stop_times.rename(columns={'stop_name': 'to_stop_name'})

# Statistiques : Temps de trajet moyen entre les stations et par mode de transport
result = stop_times.groupby(['from_stop_name', 'to_stop_name', 'route_type'])['travel_time'].mean().reset_index()
result['travel_time_min'] = result['travel_time'] / 60

# Type d'itinéraire lisible  (0=Tram, 3=Bus, etc.)
route_type_map = {
    0: 'Tram', 1: 'Metro', 2: 'Rail', 3: 'Bus', 4: 'Ferry', 5: 'Cable car',
    6: 'Gondola', 7: 'Funicular'
}
result['route_type'] = result['route_type'].map(route_type_map)


print(result.head())

  from_stop_name             to_stop_name route_type  travel_time  \
0    23 Novembre          Illkirch Mairie        Bus   162.234637   
1    23 Novembre                  Pigeons        Bus    48.330275   
2      A l'Aigle                Abattoirs        Bus     7.500000   
3      A l'Aigle         Holtzheim Centre        Bus    60.000000   
4      A l'Aigle  Holtzheim Centre Marché        Bus     0.000000   

   travel_time_min  
0         2.703911  
1         0.805505  
2         0.125000  
3         1.000000  
4         0.000000  


In [ ]:
result.to_csv('tram_bus_travel_times.csv', index=False, encoding='utf-8-sig')

## Nettoyer les lignes (aller-retour >> unique)

In [ ]:
# Charger les données
df = pd.read_csv('tram_bus_travel_times.csv')

# Créer une clé OD non directionnelle (ordre alphabétique) + mode de transport
df['od_key'] = df.apply(lambda row:
    '_'.join(sorted([row['from_stop_name'], row['to_stop_name']])), axis=1)
df['od_mode_key'] = df['od_key'] + "_" + df['route_type']

# Regrouper par cette clé
df_grouped = df.groupby('od_mode_key').agg({
    'from_stop_name': 'first',
    'to_stop_name': 'first',
    'route_type': 'first',
    'travel_time_min': 'mean'
}).reset_index(drop=True)

# Arrondir proprement
df_grouped['travel_time_min'] = df_grouped['travel_time_min'].round(1)

# Sauvegarder
df_grouped.to_csv('trajets_OD_regroupes.csv', index=False, encoding='utf-8-sig')
print("✅ Fichier enregistré : trajets_OD_regroupes.csv")


✅ Fichier enregistré : trajets_OD_regroupes.csv


In [ ]:
df_grouped

,from_stop_name,to_stop_name,route_type,travel_time_min
0,23 Novembre,Illkirch Mairie,Bus,2.4
1,23 Novembre,Pigeons,Bus,0.6
2,A l'Aigle,Abattoirs,Bus,0.6
3,A l'Aigle,Holtzheim Centre Marché,Bus,0.5
4,A l'Aigle,Holtzheim Centre,Bus,1.0
...,...,...,...,...
790,Vendenheim Gare,Vendenheim Mairie,Bus,1.5
791,Vendenheim Mairie,Vendenheim au Tilleul,Bus,0.8
792,Verdun,Wagner,Bus,0.9
793,Wagner,Ypres,Bus,1.0


## Joindre les cordonnées des arrêts

nettoyer les données de stops

In [ ]:
stops

,stop_id,stop_code,stop_lat,stop_lon,stop_name,stop_url,stop_desc
0,23NOV_01,704A,48.533146,7.706401,23 Novembre,http://www.cts-strasbourg.eu/fr/se-deplacer/fi...,NaN
1,23NOV_02,704B,48.533181,7.706580,23 Novembre,http://www.cts-strasbourg.eu/fr/se-deplacer/fi...,NaN
2,ABATT_01,10A,48.556985,7.653061,Abattoirs,http://www.cts-strasbourg.eu/fr/se-deplacer/fi...,NaN
3,ABATT_02,10B,48.556897,7.653733,Abattoirs,http://www.cts-strasbourg.eu/fr/se-deplacer/fi...,NaN
4,ABBAT_01,11A,48.491786,7.715969,Abbatiale,http://www.cts-strasbourg.eu/fr/se-deplacer/fi...,NaN
...,...,...,...,...,...,...,...
1407,ZOCOM_02,676B,48.656795,7.721799,Zone Commerciale Nord,http://www.cts-strasbourg.eu/fr/se-deplacer/fi...,NaN
1408,ZORNN_01,677A,48.609316,7.757596,Zorn,http://www.cts-strasbourg.eu/fr/se-deplacer/fi...,NaN
1409,ZORNN_02,677B,48.609197,7.757868,Zorn,http://www.cts-strasbourg.eu/fr/se-deplacer/fi...,NaN
1410,ZURIC_01,754A,48.579613,7.756006,Zurich,http://www.cts-strasbourg.eu/fr/se-deplacer/fi...,NaN


In [ ]:
# Charger le fichier stops.txt
stops = pd.read_csv('stops.txt')

# Nettoyage basique : supprimer les doublons et calculer les coordonnées moyennes par nom d'arrêt
stops_grouped = stops.groupby('stop_name', as_index=False).agg({
    'stop_lat': 'mean',
    'stop_lon': 'mean'
})

# Sauvegarder le fichier nettoyé
stops_grouped.to_csv('stops_grouped.csv', index=False, encoding='utf-8-sig')
print("✅ Fichier enregistré : stops_grouped.csv")

✅ Fichier enregistré : stops_grouped.csv


In [ ]:
stops_grouped

,stop_name,stop_lat,stop_lon
0,23 Novembre,48.533164,7.706490
1,A l'Aigle,48.557074,7.646303
2,Abattoirs,48.556941,7.653397
3,Abbatiale,48.491847,7.716050
4,Abbé Lemire,48.578144,7.724644
...,...,...,...
586,Ypres,48.586179,7.780685
587,ZAC des Maréchaux,48.631984,7.716979
588,Zone Commerciale Nord,48.656724,7.721946
589,Zorn,48.609256,7.757732


In [ ]:
# Charger les trajets et les coordonnées nettoyées
trajets = pd.read_csv('trajets_OD_regroupes.csv')
stops_grouped = pd.read_csv('stops_grouped.csv')

# Préparer les coordonnées de départ
stops_from = stops_grouped.rename(columns={
    'stop_name': 'from_stop_name',
    'stop_lat': 'from_lat',
    'stop_lon': 'from_lon'
})

# Préparer les coordonnées d’arrivée
stops_to = stops_grouped.rename(columns={
    'stop_name': 'to_stop_name',
    'stop_lat': 'to_lat',
    'stop_lon': 'to_lon'
})

# Joindre les coordonnées de départ
trajets = trajets.merge(stops_from, on='from_stop_name', how='left')

# Joindre les coordonnées d’arrivée
trajets = trajets.merge(stops_to, on='to_stop_name', how='left')

# Sauvegarder le fichier final
trajets.to_csv('trajets_OD_regroupes_coordonnees.csv', index=False, encoding='utf-8-sig')
print("✅ Coordonnées ajoutées et fichier enregistré : trajets_OD_regroupes_coordonnees.csv")


✅ Coordonnées ajoutées et fichier enregistré : trajets_OD_regroupes_coordonnees.csv


In [ ]:
trajets

,from_stop_name,to_stop_name,route_type,travel_time_min,from_lat,from_lon,to_lat,to_lon
0,23 Novembre,Illkirch Mairie,Bus,2.4,48.533164,7.706490,48.529689,7.710994
1,23 Novembre,Pigeons,Bus,0.6,48.533164,7.706490,48.534985,7.708811
2,A l'Aigle,Abattoirs,Bus,0.6,48.557074,7.646303,48.556941,7.653397
3,A l'Aigle,Holtzheim Centre Marché,Bus,0.5,48.557074,7.646303,48.557046,7.642035
4,A l'Aigle,Holtzheim Centre,Bus,1.0,48.557074,7.646303,48.557732,7.643150
...,...,...,...,...,...,...,...,...
790,Vendenheim Gare,Vendenheim Mairie,Bus,1.5,48.666087,7.717887,48.665353,7.711456
791,Vendenheim Mairie,Vendenheim au Tilleul,Bus,0.8,48.665353,7.711456,48.668811,7.710377
792,Verdun,Wagner,Bus,0.9,48.588028,7.773627,48.587081,7.776681
793,Wagner,Ypres,Bus,1.0,48.587081,7.776681,48.586179,7.780685


## Créer le csv selon mode

In [ ]:
# Charger le fichier avec coordonnées
df = pd.read_csv('trajets_OD_regroupes_coordonnees.csv')

# Arrondir les temps de trajet à 1 décimale
df['travel_time_min'] = df['travel_time_min'].round(1)

# Trouver tous les types de transport présents
modes = df['route_type'].unique()

# Pour chaque mode, créer un fichier CSV distinct
for mode in modes:
    df_mode = df[df['route_type'] == mode].copy()
    filename = f"trajets_{mode.lower()}.csv"
    df_mode.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ Fichier créé : {filename}")


✅ Fichier créé : trajets_bus.csv
✅ Fichier créé : trajets_tram.csv


# Spatialiser sur la carte (facultif pour avoir une image, pas nécessaire de visualiser)

## Cartographier les arrêts (POI base)

In [ ]:
# Charger les données de tramway
df = pd.read_csv('trajets_tram.csv')

# Extraire les arrêts de départ et d’arrivée
from_stops = df[['from_stop_name', 'from_lat', 'from_lon']].rename(columns={
    'from_stop_name': 'stop_name',
    'from_lat': 'lat',
    'from_lon': 'lon'
})

to_stops = df[['to_stop_name', 'to_lat', 'to_lon']].rename(columns={
    'to_stop_name': 'stop_name',
    'to_lat': 'lat',
    'to_lon': 'lon'
})

# Fusionner les deux, puis retirer les doublons
tram_stops = pd.concat([from_stops, to_stops]).drop_duplicates(subset='stop_name')

# Créer une carte centrée sur le centre des points
m = folium.Map(location=[tram_stops['lat'].mean(), tram_stops['lon'].mean()], zoom_start=13)

# Ajouter les points
for _, row in tram_stops.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['stop_name'],
        icon=folium.Icon(color='blue', icon='train', prefix='fa')
    ).add_to(m)

m

In [ ]:
#### bus just pour l'essai

# Charger les données de bus
df = pd.read_csv('trajets_bus.csv')

# Extraire les arrêts de départ et d’arrivée
from_stops = df[['from_stop_name', 'from_lat', 'from_lon']].rename(columns={
    'from_stop_name': 'stop_name',
    'from_lat': 'lat',
    'from_lon': 'lon'
})

to_stops = df[['to_stop_name', 'to_lat', 'to_lon']].rename(columns={
    'to_stop_name': 'stop_name',
    'to_lat': 'lat',
    'to_lon': 'lon'
})

# Fusionner les deux, puis retirer les doublons
bus_stops = pd.concat([from_stops, to_stops]).drop_duplicates(subset='stop_name')

# Créer une carte centrée
m = folium.Map(location=[bus_stops['lat'].mean(), bus_stops['lon'].mean()], zoom_start=12)

# Ajouter les arrêts avec une icône différente (bus orange)
for _, row in bus_stops.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['stop_name'],
        icon=folium.Icon(color='orange', icon='bus', prefix='fa')
    ).add_to(m)

m


## Cartographier les segments

In [ ]:
# Charger les trajets tram
df = pd.read_csv('trajets_tram.csv')

# Créer la carte centrée
m = folium.Map(location=[df['from_lat'].mean(), df['from_lon'].mean()], zoom_start=13)

# Ajouter les arrets en bleu
arrets = pd.concat([
    df[['from_stop_name', 'from_lat', 'from_lon']].rename(columns={
        'from_stop_name': 'stop_name',
        'from_lat': 'lat',
        'from_lon': 'lon'
    }),
    df[['to_stop_name', 'to_lat', 'to_lon']].rename(columns={
        'to_stop_name': 'stop_name',
        'to_lat': 'lat',
        'to_lon': 'lon'
    })
]).drop_duplicates(subset='stop_name')

for _, row in arrets.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['stop_name'],
        icon=folium.Icon(color='blue', icon='train', prefix='fa')
    ).add_to(m)

# Ajouter les segments (lignes entre arrêts connectés)
for _, row in df.iterrows():
    folium.PolyLine(
        locations=[
            [row['from_lat'], row['from_lon']],
            [row['to_lat'], row['to_lon']]
        ],
        color='blue',
        weight=4,
        opacity=0.7,
        tooltip=f"{row['from_stop_name']} → {row['to_stop_name']} ({row['travel_time_min']} min)"
    ).add_to(m)

m


# **2. ORS**

5.B.2. Calcul des temps à pied, à vélo et en voiture

## Installer bibliothèque

In [ ]:
!pip install openrouteservice
import openrouteservice
from openrouteservice import convert

## Calculer les temps du trajet (pieds, vélo, voiture)

In [ ]:
import pandas as pd
import openrouteservice
import numpy as np
from itertools import product

# === Paramètres ===
ORS_API_KEY = "#########" #faut s'y inscrire (gratuit) et mettre la clé
MAX_SIZE = 59  # max 59x59 = 3481
PROFILES = ['foot-walking', 'cycling-regular', 'driving-car']

# === ORS client ===
client = openrouteservice.Client(key=ORS_API_KEY)

# === Charger les données ===
df = pd.read_csv("trajets_tram.csv")
coord_df = pd.DataFrame({
    'stop_name': df['from_stop_name'].tolist() + df['to_stop_name'].tolist(),
    'lat': df['from_lat'].tolist() + df['to_lat'].tolist(),
    'lon': df['from_lon'].tolist() + df['to_lon'].tolist()
}).drop_duplicates().dropna().reset_index(drop=True)

coord_df = coord_df.drop_duplicates(subset=['stop_name']).reset_index(drop=True)
coords = list(zip(coord_df['lon'], coord_df['lat']))  # ORS format

# === Fonction de découpage en blocs ORS-safe ===
def split_blocks(data, size):
    for i in range(0, len(data), size):
        for j in range(0, len(data), size):
            sources = list(range(i, min(i+size, len(data))))
            dests = list(range(j, min(j+size, len(data))))
            yield sources, dests

# === Fonction de requête ORS matrix ===
def get_matrix_sub(client, coords, sources, destinations, profile):
    matrix = client.distance_matrix(
        locations=coords,
        profile=profile,
        metrics=['duration'],
        sources=sources,
        destinations=destinations,
        resolve_locations=False
    )
    return np.array(matrix['durations'])

# === Calculer matrices par profil ===
matrices = {}
for profile in PROFILES:
    print(f"Calcul de la matrice pour : {profile}")
    full_matrix = np.full((len(coords), len(coords)), np.nan)

    for sources, destinations in split_blocks(coords, MAX_SIZE):
        try:
            sub_matrix = get_matrix_sub(client, coords, sources, destinations, profile)
            for si, s in enumerate(sources):
                for di, d in enumerate(destinations):
                    full_matrix[s, d] = sub_matrix[si, di]
        except Exception as e:
            print(f"Erreur sur bloc {sources[0]}-{sources[-1]} x {destinations[0]}-{destinations[-1]} : {e}")

    matrices[profile] = pd.DataFrame(
        full_matrix,
        index=coord_df['stop_name'],
        columns=coord_df['stop_name']
    )

# === Sauvegarde des résultats ===
for profile in PROFILES:
    filename = f"matrix_durations_{profile.replace('-', '_')}.csv"
    matrices[profile].to_csv(filename, encoding='utf-8-sig')
    print(f"✅ Matrice enregistrée : {filename}")


Calcul de la matrice pour : foot-walking
Calcul de la matrice pour : cycling-regular
Calcul de la matrice pour : driving-car
✅ Matrice enregistrée : matrix_durations_foot_walking.csv
✅ Matrice enregistrée : matrix_durations_cycling_regular.csv
✅ Matrice enregistrée : matrix_durations_driving_car.csv


# **3.Mise en ensemble & Export**

5.B.3. Mise en ensemble dans un tableau et Export des résultats

## fusionner des tableaux

In [ ]:
# --- Charger les trajets tram ---
df = pd.read_csv('trajets_tram.csv')

# --- Charger les matrices ORS ---
df_foot = pd.read_csv('matrix_durations_foot_walking.csv', index_col=0)
df_bike = pd.read_csv('matrix_durations_cycling_regular.csv', index_col=0)
df_car = pd.read_csv('matrix_durations_driving_car.csv', index_col=0)

# --- Arrondir les matrices en minutes ---
df_foot = df_foot / 60
df_bike = df_bike / 60
df_car = df_car / 60

# --- Fonction pour récupérer les durées à partir des matrices ---
# --- moyenne symétrique ---
def get_symmetric_mean_duration(row, matrix):
    from_stop = row['from_stop_name']
    to_stop = row['to_stop_name']
    try:
        val1 = matrix.loc[from_stop, to_stop]
    except KeyError:
        val1 = None
    try:
        val2 = matrix.loc[to_stop, from_stop]
    except KeyError:
        val2 = None

    if pd.notna(val1) and pd.notna(val2):
        return round((val1 + val2) / 2, 1)
    elif pd.notna(val1):
        return round(val1, 1)
    elif pd.notna(val2):
        return round(val2, 1)
    else:
        return None

# --- Ajouter les colonnes ---
df['pieds_min'] = df.apply(lambda row: get_symmetric_mean_duration(row, df_foot), axis=1)
df['velo_min'] = df.apply(lambda row: get_symmetric_mean_duration(row, df_bike), axis=1)
df['voiture_min'] = df.apply(lambda row: get_symmetric_mean_duration(row, df_car), axis=1)

# --- Renommer la colonne tram pour cohérence ---
df = df.rename(columns={'travel_time_min': 'tram_min'})

# --- Réorganiser les colonnes ---
df_final = df[['from_stop_name', 'to_stop_name', 'route_type',
               'from_lat', 'from_lon', 'to_lat', 'to_lon',
               'tram_min', 'pieds_min', 'velo_min', 'voiture_min']]

# --- Ajouter une colonne ID (index numérique) ---
df_final.insert(0, 'id', range(1, len(df_final) + 1))

# --- Export final ---
df_final.to_csv('trajets_tram_comparaison_modes.csv', index=False, encoding='utf-8-sig')
print("✅ Fichier enrichi enregistré : trajets_tram_comparaison_modes.csv")


✅ Fichier enrichi enregistré : trajets_tram_comparaison_modes.csv


In [ ]:
df_final

,id,from_stop_name,to_stop_name,route_type,from_lat,from_lon,to_lat,to_lon,tram_min,pieds_min,velo_min,voiture_min
0,1,Alt Winmärik-Vieux Marché aux Vins,Faubourg National,Tram,48.583377,7.740221,48.582399,7.736567,2.0,5.3,3.1,2.7
1,2,Alt Winmärik-Vieux Marché aux Vins,Homme de Fer,Tram,48.583377,7.740221,48.584190,7.744681,2.0,4.2,2.8,5.2
2,3,Ancienne Synagogue / Les Halles,Gare Centrale,Tram,48.585303,7.742310,48.585078,7.735309,2.0,7.5,2.6,4.4
3,4,Ancienne Synagogue / Les Halles,Homme de Fer,Tram,48.585303,7.742310,48.584190,7.744681,2.0,3.0,1.4,3.6
4,5,Aristide Briand,Citadelle,Tram,48.568567,7.776870,48.571436,7.780063,1.5,5.3,2.0,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...
84,85,Parc du Contades,République,Tram,48.590298,7.755962,48.586524,7.754262,1.7,6.1,2.1,1.5
85,86,Parlement Européen,Wacken,Tram,48.599336,7.766790,48.598525,7.761026,1.5,5.9,2.0,1.3
86,87,Port du Rhin,Starcoop,Tram,48.572552,7.795009,48.574136,7.785681,1.5,8.8,3.0,3.9
87,88,Rives de l'Aar,Wacken,Tram,48.600492,7.754451,48.598525,7.761026,1.5,7.6,2.3,1.7


## cartographier

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import folium

# --- Charger le fichier enrichi ---pm
df = pd.read_csv("trajets_tram_comparaison_modes.csv")

# --- Créer les géométries des trajets (ligne de from → to) ---
df['geometry'] = df.apply(lambda row: LineString([
    (row['from_lon'], row['from_lat']),
    (row['to_lon'], row['to_lat'])
]), axis=1)

gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

# --- Créer la carte centrée automatiquement sur Strasbourg ---
centroid = gdf.geometry.unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=13)

# --- Couleur unique ou par type (ici simplifiée) ---
color = 'blue'

# --- Ajouter les segments avec pop-up ---
for _, row in gdf.iterrows():
    popup_text = (
        f"<b>Départ :</b> {row['from_stop_name']}<br>"
        f"<b>Arrivée :</b> {row['to_stop_name']}<br>"
        f"<b>Tram :</b> {row['tram_min']} min<br>"
        f"<b>Pied :</b> {row['pieds_min']} min<br>"
        f"<b>Vélo :</b> {row['velo_min']} min<br>"
        f"<b>Voiture :</b> {row['voiture_min']} min"
    )

    folium.GeoJson(
        row.geometry,
        style_function=lambda x: {'color': color, 'weight': 4},
        tooltip=popup_text
    ).add_to(m)

# --- Ajouter les arrêts avec des cercles noirs ---
stops = pd.concat([
    df[['from_stop_name', 'from_lat', 'from_lon']],
    df[['to_stop_name', 'to_lat', 'to_lon']].rename(columns={
        'to_stop_name': 'from_stop_name', 'to_lat': 'from_lat', 'to_lon': 'from_lon'
    })
]).drop_duplicates()

for _, row in stops.iterrows():
    folium.CircleMarker(
        location=[row['from_lat'], row['from_lon']],
        radius=4,
        color='black',
        fill=True,
        fill_color='black',
        popup=row['from_stop_name']
    ).add_to(m)

# --- Ajouter les arrêts avec des icônes tramway ---
for _, row in stops.iterrows():
    folium.Marker(
        location=[row['from_lat'], row['from_lon']],
        icon=folium.Icon(icon='tram', prefix='fa', color='blue'),
        popup=row['from_stop_name']
    ).add_to(m)

m


/tmp/ipython-input-7-3021070066.py:18: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = gdf.geometry.unary_union.centroid


## Exporter en shp

In [ ]:
gdf.to_file("segments_tram_comparaison.shp", encoding='utf-8')

/tmp/ipython-input-8-595846813.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("segments_tram_comparaison.shp", encoding='utf-8')
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'from_stop_name' to 'from_stop_'
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'to_stop_name' to 'to_stop_na'
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'voiture_min' to 'voiture_mi'
  ogr_write(


In [ ]:
gdf_stops = gpd.GeoDataFrame(
    stops,
    geometry=gpd.points_from_xy(stops['from_lon'], stops['from_lat']),
    crs="EPSG:4326"
)
gdf_stops.to_file("arrets_tramway.shp", encoding='utf-8')

/tmp/ipython-input-9-348186740.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_stops.to_file("arrets_tramway.shp", encoding='utf-8')
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'from_stop_name' to 'from_stop_'
  ogr_write(


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Charger les données
df = pd.read_csv('trajets_bus.csv')

# Préparer les arrêts
from_stops = df[['from_stop_name', 'from_lat', 'from_lon']].rename(columns={
    'from_stop_name': 'stop_name',
    'from_lat': 'lat',
    'from_lon': 'lon'
})
to_stops = df[['to_stop_name', 'to_lat', 'to_lon']].rename(columns={
    'to_stop_name': 'stop_name',
    'to_lat': 'lat',
    'to_lon': 'lon'
})
bus_stops = pd.concat([from_stops, to_stops]).drop_duplicates(subset='stop_name')

# Convertir en GeoDataFrame
bus_stops['geometry'] = bus_stops.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
gdf = gpd.GeoDataFrame(bus_stops, geometry='geometry', crs='EPSG:4326')

# Exporter en Shapefile
gdf.to_file('arrets_bus.shp', driver='ESRI Shapefile')